## Data Preprocessing for OCR Results

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
import torch
print(torch.cuda.is_available())

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/train_data_noise120k.csv')

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

# Preprocess data
def preprocess_data(row):
    # Combine concatenated_text and entity_name
    input_text = f"{row['entity_name']} ? {row['concatenated_text']}"
    # Target value is the entity_value
    target_text = row['entity_value']
    return {"input_text": input_text, "target_text": target_text}

# Apply preprocessing
processed_df = df.apply(preprocess_data, axis=1)
processed_df = pd.DataFrame(processed_df.tolist())

# Split data into train and validation
train_df, val_df = train_test_split(processed_df, test_size=0.1, random_state=42)


In [ ]:
# Convert DataFrame to Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize and encode the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples['target_text'], max_length=64, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

## Training T5 Entity Model

In [ ]:
# Define training arguments with GPU settings
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='./results',
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"  # Optional: Disable W&B logging
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("t5_entity_value_model")
tokenizer.save_pretrained("t5_entity_value_model")

## Generating Predictions (Test.csv)

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
from torch.utils.data import DataLoader
import torch

# Load the test data (without target/entity_value)
test_df = pd.read_csv('/datasets/test.csv')  # Adjust the path to your test.csv

# Preprocess data for the test dataset
def preprocess_test_data(row):
    # Combine entity_name and text for input_text
    input_text = f"{row['entity_name']} ? {row['text']}"
    return {"input_text": input_text}

# Apply preprocessing to the test dataset
processed_test_df = test_df.apply(preprocess_test_data, axis=1)
processed_test_df = pd.DataFrame(processed_test_df.tolist())

# Convert DataFrame to Dataset format
test_dataset = Dataset.from_pandas(processed_test_df)

# Specify the directory where the model and tokenizer are saved
saved_model_dir = "t5_entity_value_model"
# Load the tokenizer and model from the saved directory
tokenizer = T5Tokenizer.from_pretrained(saved_model_dir)
model = T5ForConditionalGeneration.from_pretrained(saved_model_dir)

# Tokenize and encode the test dataset
def tokenize_test_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    return inputs

# Apply tokenization to the test dataset
test_dataset = test_dataset.map(tokenize_test_function, batched=True)

# Convert Dataset to PyTorch DataLoader format
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids']) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask}

batch_size = 32  # Adjust the batch size as needed
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Generate predictions in batches
test_predictions = []
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation for faster processing
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Generate predictions
        outputs = model.generate(input_ids, attention_mask=attention_mask)

        # Decode the predicted output
        predicted_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

        # Post-process predictions to replace "#" with an empty string
        predicted_texts = [text.replace("#", "") for text in predicted_texts]

        # Append batch predictions to the final list
        test_predictions.extend(predicted_texts)

# Store the predictions in the original test DataFrame
test_df['prediction'] = test_predictions


# Save the predictions to CSV
test_df[['index', 'prediction']].to_csv('test_predictions.csv', index=False)